In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


In [2]:
# First, load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload all modules
%autoreload 2

This notebook demostrate how llm uses functions in tools/remote_tools.py to ping user specify api and the get the expected answer

In [6]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..')))

from langchain_openai import ChatOpenAI
from typing import Annotated, TypedDict

from core.agents.tools.remote_tools import remote_tool_func, fetch_tools, remote_api_call,construct_url
from langchain.agents import Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_openai_tools_agent


MODEL = "gpt-3.5-turbo-0125"
TEMPERATURE = 0

In [7]:
fetch_tool = Tool(
    name="fetch_remote_tool",
    func=fetch_tools,
    description="Fetch user registered remote tools by user id. Returns a list of tool dictionaries."
)

ping_tool = Tool(
    name="ping_remote_api",
    func=remote_api_call,
    description="Ping a constructed API URL and return the response."
)

construct_url_tool = Tool(
    name="construct_url",
    func=construct_url,
    description="Construct a full URL from a tool dictionary. Input must be a dictionary in the format {'name': string, 'url': string, 'params': dict}."
)

tools = [fetch_tool, ping_tool,construct_url_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a smart API assistant. "
     "1. First, use 'fetch_remote_tool' with user_id to retrieve a list of tool dictionaries. "
     "2. Then, select the correct tool (e.g., 'temperature_api'). "
     "3. Pass the **entire tool dictionary** (not just the URL) to 'construct_url' to generate a full URL. "
     "4. Finally, use 'ping_remote_api' with the fully constructed URL to get the API response. "
     "Be sure to pass dictionaries, not strings, when constructing URLs."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")
])

# model = ChatOllama(model="mistral-nemo", temperature=0)
model  = ChatOpenAI(
    model=MODEL, 
    temperature=TEMPERATURE,
    # base_url="http://host.docker.internal:11434", # if running in the studio
    )
agent = create_openai_tools_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "you have user_id 123. what is the temperature?"})

print(f"Agent result: {result}")




> Entering new AgentExecutor chain...

Invoking: `fetch_remote_tool` with `123`


[{'name': 'temperature_api', 'url': 'https://temperature/api?', 'params': {'key': 'whatever'}}, {'name': 'news', 'url': 'https://nytimes/api', 'params': 'whatever'}]
Invoking: `construct_url` with `{"name": "temperature_api", "url": "https://temperature/api?", "params": {"key": "whatever"}}`


https://temperature/api?key=whatever
Invoking: `ping_remote_api` with `https://temperature/api?key=whatever`


{'temperature': 20}
Invoking: `fetch_remote_tool` with `123`


[{'name': 'temperature_api', 'url': 'https://temperature/api?', 'params': {'key': 'whatever'}}, {'name': 'news', 'url': 'https://nytimes/api', 'params': 'whatever'}]
Invoking: `fetch_remote_tool` with `123`


[{'name': 'temperature_api', 'url': 'https://temperature/api?', 'params': {'key': 'whatever'}}, {'name': 'news', 'url': 'https://nytimes/api', 'params': 'whatever'}]
Invoking: `fetch_remote_tool` with `123`


[{'name': 'temperature_api', 